In [1]:
import os

# Must be set before any CUDA context is created
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

import torch

# Determinism / precision
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False
torch.set_float32_matmul_precision("highest")
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True)


In [2]:
# -------------------------
# 0) CONFIG (edit these)
# -------------------------
import os, sys

# Project code (must contain the csiro package)
CSIRO_CODE_DIR = "/notebooks/CSIRO"

# DINOv3 repo/code dir (must contain hubconf.py)
DINO_REPO = "/notebooks/dinov3"

# DINOv3 pretrained backbone weights (.pth)
DINO_WEIGHTS = "/notebooks/kaggle/csiro/weights/dinov3/dinov3_vitb16_pretrain.pth"

# Ensemble checkpoint produced by CV (dict with key 'states')
WEIGHTS_PATH = "/notebooks/kaggle/csiro/weights/cv5_v1_f0a.pt"

# Competition data
COMP_ROOT = "/notebooks/kaggle/csiro/"
TEST_CSV = f"{COMP_ROOT}/test.csv"
# IMPORTANT: test.csv image_path values look like "test/IDxxxx.jpg", so IMAGE_ROOT should be COMP_ROOT
IMAGE_ROOT = COMP_ROOT

# Inference params
IMG_SIZE = 512
BATCH_SIZE = 64
NUM_WORKERS = 2
DEVICE = "cuda"  # or "cpu"

# TTA / ensemble knobs
TTA_ROT90 = True
TTA_AGG = "mean"
ENS_AGG = "mean"

OUTPUT_PATH = "/notebooks/kaggle/csiro/sub/submission.csv"

# --- Env vars expected by csiro.config (no defaults) ---
os.environ["TORCH_HOME"] = "notebooks/kaggle/working/torch_home"
os.environ["DINO_WB"] = "https://dinov3.llamameta.net/dinov3_vitb16/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoidW84aXJvdGQyeThwcGpuNXFveGthZTE4IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZGlub3YzLmxsYW1hbWV0YS5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjU5NzI4MTd9fX1dfQ__&Signature=H5H5kLVc6V83i-s2euNHx6t9KlVeG27QKX6qtkXNiLwEzuCshJD4RfwUbQv8oBJOZXPezAVJZPRkYRdsb4jh-LQ72DZtEuNkjNKHf7Pn57wzee0bjEYjWdJmOqK4waaSe9TQqELM%7EPgzdAT4LCSHYcFQ%7EleRnHGGGJiHBmTd6e1xZYhvUCfkvVD1TG-zM7R0-P%7EMLetHMvWl%7EUapCMYthsWqZctsYAQKUQxsLrly8Y4EaM8hm5nowpArPZC4myNO1iiXld5Hc3t9CVLEdYT7LIct0x6cf3-B-6WOgxGb7LdLPCcZPPfoGgX3KGtTAgNQYOpGFs-hgILFHRKVOJ7T3A__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1893388161261111"
os.environ["DINO_WL"] = "https://dinov3.llamameta.net/dinov3_vitl16/dinov3_vitl16_pretrain_lvd1689m-8aa4cbdd.pth?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoidW84aXJvdGQyeThwcGpuNXFveGthZTE4IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZGlub3YzLmxsYW1hbWV0YS5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjU5NzI4MTd9fX1dfQ__&Signature=H5H5kLVc6V83i-s2euNHx6t9KlVeG27QKX6qtkXNiLwEzuCshJD4RfwUbQv8oBJOZXPezAVJZPRkYRdsb4jh-LQ72DZtEuNkjNKHf7Pn57wzee0bjEYjWdJmOqK4waaSe9TQqELM%7EPgzdAT4LCSHYcFQ%7EleRnHGGGJiHBmTd6e1xZYhvUCfkvVD1TG-zM7R0-P%7EMLetHMvWl%7EUapCMYthsWqZctsYAQKUQxsLrly8Y4EaM8hm5nowpArPZC4myNO1iiXld5Hc3t9CVLEdYT7LIct0x6cf3-B-6WOgxGb7LdLPCcZPPfoGgX3KGtTAgNQYOpGFs-hgILFHRKVOJ7T3A__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1893388161261111"
os.environ["DINO_WL_plus"] = "https://dinov3.llamameta.net/dinov3_vith16plus/dinov3_vith16plus_pretrain_lvd1689m-7c1da9a5.pth?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoidW84aXJvdGQyeThwcGpuNXFveGthZTE4IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZGlub3YzLmxsYW1hbWV0YS5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjU5NzI4MTd9fX1dfQ__&Signature=H5H5kLVc6V83i-s2euNHx6t9KlVeG27QKX6qtkXNiLwEzuCshJD4RfwUbQv8oBJOZXPezAVJZPRkYRdsb4jh-LQ72DZtEuNkjNKHf7Pn57wzee0bjEYjWdJmOqK4waaSe9TQqELM%7EPgzdAT4LCSHYcFQ%7EleRnHGGGJiHBmTd6e1xZYhvUCfkvVD1TG-zM7R0-P%7EMLetHMvWl%7EUapCMYthsWqZctsYAQKUQxsLrly8Y4EaM8hm5nowpArPZC4myNO1iiXld5Hc3t9CVLEdYT7LIct0x6cf3-B-6WOgxGb7LdLPCcZPPfoGgX3KGtTAgNQYOpGFs-hgILFHRKVOJ7T3A__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1893388161261111"
os.environ["DEFAULT_DINO_REPO_DIR"] = DINO_REPO
os.environ["DEFAULT_DATA_ROOT"] = COMP_ROOT

sys.path.insert(0, CSIRO_CODE_DIR)


In [3]:
# -------------------------
# 1) Imports
# -------------------------
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset
from PIL import Image

import csiro
from csiro.config import TARGETS, dino_hub_name, DEFAULT_MODEL_SIZE, DEFAULT_PLUS
from csiro.transforms import PadToSquare, post_tfms
from csiro.eval import predict_ensemble

print("TARGETS:", TARGETS)

TARGETS: ['Dry_Green_g', 'Dry_Clover_g', 'Dry_Dead_g', 'GDM_g', 'Dry_Total_g']


In [7]:
from pathlib import Path
# Create checkpoints dir
ckpt_dir = Path(torch.hub.get_dir()) / "checkpoints"
ckpt_dir.mkdir(parents=True, exist_ok=True)

# Symlink the weight file from /kaggle/input (read-only) into the hub cache
src = Path(DINO_WEIGHTS)                  # /kaggle/input/.../dinov3_vitb16_pretrain.pth
dst = ckpt_dir / src.name                 # /kaggle/working/torch_hub/checkpoints/dinov3_vitb16_pretrain.pth

if not dst.exists():
    dst.symlink_to(src)

In [4]:
# -------------------------
# 2) Load checkpoint + backbone
# -------------------------
ckpt = torch.load(WEIGHTS_PATH, map_location="cpu", weights_only=False)
if isinstance(ckpt, dict) and "states" in ckpt:
    states = ckpt["states"]
else:
    states = ckpt

sys.path.insert(0, DINO_REPO)
backbone = torch.hub.load(
    DINO_REPO,
    dino_hub_name(model_size=str(DEFAULT_MODEL_SIZE), plus=str(DEFAULT_PLUS)),
    source="local",
    weights=DINO_WEIGHTS,
).cuda()

backbone.eval()
print("Loaded states", type(states))


Loaded states <class 'list'>


In [6]:
from csiro.eval import load_dinov3_regressor_from_pt as load_model
from csiro.eval import forward_trainable_random


model = load_model(WEIGHTS_PATH, backbone=backbone)
forward_trainable_random(model, seed=420, device="cuda", head_only=False)

[tensor([[ 3.9159, -0.5923,  2.9785,  3.7281,  4.3917]], device='cuda:0')]

In [8]:
# -------------------------
# 3) Read test.csv (long format)
# -------------------------
df = pd.read_csv(TEST_CSV)
print("test.csv columns:", list(df.columns))
print(df.head(3))

IMAGE_PATH_COL = "image_path"
TARGET_NAME_COL = "target_name"
SAMPLE_ID_COL = "sample_id"

if IMAGE_PATH_COL not in df.columns:
    raise ValueError(f"Expected column {IMAGE_PATH_COL} in test.csv")
if TARGET_NAME_COL not in df.columns:
    raise ValueError(f"Expected column {TARGET_NAME_COL} in test.csv")

df_img = df.drop_duplicates(subset=[IMAGE_PATH_COL]).reset_index(drop=True)
print("rows (long):", len(df), "unique images:", len(df_img))


test.csv columns: ['sample_id', 'image_path', 'target_name']
                    sample_id             image_path   target_name
0  ID1001187975__Dry_Clover_g  test/ID1001187975.jpg  Dry_Clover_g
1    ID1001187975__Dry_Dead_g  test/ID1001187975.jpg    Dry_Dead_g
2   ID1001187975__Dry_Green_g  test/ID1001187975.jpg   Dry_Green_g
rows (long): 5 unique images: 1


In [9]:
# -------------------------
# 4) Dataset + inference
# -------------------------
class TestDataset(Dataset):
    def __init__(self, df, root, img_col, img_size):
        self.df = df.reset_index(drop=True)
        self.root = root
        self.img_col = img_col
        self.pre = T.Compose([
            T.Lambda(lambda im: im.convert("RGB")),
            PadToSquare(fill=0),
            T.Resize((int(img_size), int(img_size)), antialias=True),
            post_tfms(),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        rel = self.df.loc[i, self.img_col]
        p = os.path.join(self.root, rel) if self.root else rel
        with Image.open(p) as im:
            x = self.pre(im)
        return x

ds = TestDataset(df_img, IMAGE_ROOT, IMAGE_PATH_COL, IMG_SIZE)
preds = predict_ensemble(
    ds,
    states=states,
    backbone=backbone,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    device=DEVICE,
    tta_rot90=TTA_ROT90,
    tta_agg=TTA_AGG,
    ens_agg=ENS_AGG,
)

preds = preds.detach().cpu()
print("preds shape:", tuple(preds.shape))


preds shape: (1, 5)


In [10]:
# -------------------------
# 5) Build submission (long format)
# -------------------------
target_to_idx = {t: i for i, t in enumerate(TARGETS)}
preds_np = preds.numpy()

pred_by_path = {
    df_img.loc[i, IMAGE_PATH_COL]: preds_np[i]
    for i in range(len(df_img))
}

if SAMPLE_ID_COL in df.columns:
    sample_ids = df[SAMPLE_ID_COL].astype(str)
else:
    image_ids = df[IMAGE_PATH_COL].apply(lambda p: os.path.splitext(os.path.basename(p))[0])
    sample_ids = image_ids + "__" + df[TARGET_NAME_COL].astype(str)

targets = []
for i, row in df.iterrows():
    p = row[IMAGE_PATH_COL]
    t_name = row[TARGET_NAME_COL]
    if t_name not in target_to_idx:
        raise ValueError(f"Unknown target_name: {t_name}")
    vec = pred_by_path[p]
    targets.append(float(vec[target_to_idx[t_name]]))

sub = pd.DataFrame({
    "sample_id": sample_ids.values,
    "target": np.asarray(targets, dtype=np.float32),
})

assert len(sub) == len(df)
assert list(sub.columns) == ["sample_id", "target"]
assert np.isfinite(sub["target"].values).all()

print("Wrote", OUTPUT_PATH)
print(sub.head(10))


Wrote /notebooks/kaggle/csiro/sub/submission.csv
                    sample_id      target
0  ID1001187975__Dry_Clover_g    0.000000
1    ID1001187975__Dry_Dead_g   18.312599
2   ID1001187975__Dry_Green_g  332.788330
3   ID1001187975__Dry_Total_g  238.660995
4         ID1001187975__GDM_g  176.639587


In [12]:

from csiro.model import DINOv3Regressor

def build_ensemble_from_states(states, backbone, device="cuda"):
    if states and isinstance(states[0], list):
        states = [s for fold in states for s in fold]

    models = []
    for s in states:
        model = DINOv3Regressor(
            backbone,
            hidden=int(s["head_hidden"]),
            drop=float(s["head_drop"]),
            depth=int(s["head_depth"]),
            num_neck=int(s["num_neck"]),
        ).to(device)

        parts = s.get("parts", s)
        for name in ("neck", "head", "norm"):
            part = getattr(model, name, None)
            if part is not None and name in parts:
                part.load_state_dict(parts[name], strict=True)

        if hasattr(model, "set_train"):
            model.set_train(False)
        model.eval()
        models.append(model)

    return models


models = build_ensemble_from_states(states, backbone, device=DEVICE)

In [18]:
def neck_head_only_output_dtype(
    model,
    *,
    feat_dim=None,
    tokens_len=197,
    device="cuda",
    seed=1234,
    dtype=torch.float32,
):
    if isinstance(model, list):
        model = model[0]
    if hasattr(model, "set_train"):
        model.set_train(False)
    model.eval()
    model = model.to(device=device, dtype=dtype)

    if feat_dim is None:
        feat_dim = int(model.feat_dim)

    g = torch.Generator(device=device).manual_seed(seed)
    x = torch.rand(
        1, int(tokens_len), int(feat_dim),
        generator=g, device=device, dtype=dtype
    )

    with torch.no_grad():
        tokens = x
        for block in model.neck:
            try:
                tokens = block(tokens, None)
            except TypeError:
                tokens = block(tokens)
        cls = tokens[:, 0, :]
        cls = model.norm(cls)
        y = model.head(cls)

    return y  # tensor in the requested dtype


out = neck_head_only_output_dtype(models, device="cpu")


In [31]:

def backbone_only_output_dtype(
    backbone,
    *,
    img_size=512,
    device="cuda",
    seed=1234,
    dtype=torch.float32,
):
    bb = backbone.to(device=device, dtype=dtype)
    bb.eval()

    g = torch.Generator(device=device).manual_seed(seed)
    x = torch.rand(1, 3, int(img_size), int(img_size), generator=g, device=device, dtype=dtype)

    with torch.no_grad():
        out = bb(x)

    return out  # tensor/dict/tuple (backbone-dependent)


out = backbone_only_output_dtype(backbone, device="cuda")

In [36]:
a = out[0]["x_norm_clstoken"][0][:30].cpu()
b = [ 0.5429,  0.9854,  0.1405, -0.7231,  0.1026, -0.3698, -0.2895, -0.0146,
         0.4019, -0.9260, -0.0975, -0.1967, -0.8500, -1.0350,  0.4145,  0.6360,
         0.4236, -0.5198,  1.2021, -0.3985,  0.7240, -0.1795, -0.5919,  1.6447,
         0.4511, -0.2003, -1.5573, -0.2953, -0.4824,  0.2522]

torch.allclose(a, torch.tensor(b), atol=1e-1)

True

In [37]:
15*6

90

In [30]:
a = [ 0.5732,  0.9725,  0.0507, -0.7856,  0.2029, -0.3223, -0.2652,  0.0990,
         0.4151, -0.9029, -0.1418, -0.2274, -0.8565, -1.0758,  0.2542,  0.5936,
         0.4904, -0.4633,  1.3237, -0.4250,  0.9271, -0.0781, -0.6784,  1.5840,
         0.4213, -0.3091, -1.5974, -0.3488, -0.4833,  0.0859]

b = [ 0.5732,  0.9725,  0.0507, -0.7856,  0.2030, -0.3223, -0.2652,  0.0990,
         0.4150, -0.9029, -0.1418, -0.2274, -0.8565, -1.0758,  0.2541,  0.5936,
         0.4904, -0.4633,  1.3237, -0.4250,  0.9271, -0.0781, -0.6784,  1.5840,
         0.4213, -0.3091, -1.5974, -0.3488, -0.4833,  0.0858]

torch.allclose(torch.tensor(a), torch.tensor(b), atol=1e-4)

True